In [ ]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 🚀 Start Timer for Sequential Execution
start_time = time.time()

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/HighPerformanceMachineLearning/CrimeDatafrom2020toPresent.csv')

# Convert DATE OCC to datetime format
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'], errors='coerce')

# Extract time features
df['Year_OCC'] = df['DATE OCC'].dt.year
df['Month_OCC'] = df['DATE OCC'].dt.month
df['Day_OCC'] = df['DATE OCC'].dt.day
df['Hour_OCC'] = df['TIME OCC'] // 100  # Convert HHMM to hours

# Count crimes per location
crime_counts = df.groupby(['LAT', 'LON']).size()

# Assign Crime Risk Scores (1-10) and scale to 1-5
df['Crime_Risk'] = df.set_index(['LAT', 'LON']).index.map(lambda x: crime_counts.get(x, 0))
df['Crime_Risk'] = np.ceil(10 * (df['Crime_Risk'] / df['Crime_Risk'].max())).astype(int)
df['Crime_Risk'] = df['Crime_Risk'].clip(1, 10)
df['Crime_Risk'] = np.ceil(df['Crime_Risk'] / 2).astype(int)

# Normalize `Crime_Risk` to 0-1
df['Crime_Risk'] = (df['Crime_Risk'] - 1) / 4

# Normalize features
features = ['Year_OCC', 'Month_OCC', 'Day_OCC', 'Hour_OCC', 'LAT', 'LON']
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

# Train-Test Split
train_df, _ = train_test_split(df, test_size=0.2, random_state=42)

# Define Dataset
class CrimeDatasetRegression(Dataset):
    def __init__(self, data):
        self.x = torch.tensor(data[features].values, dtype=torch.float32)
        self.y = torch.tensor(data['Crime_Risk'].values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Create DataLoader (Single-Threaded, No Parallelism)
train_dataset = CrimeDatasetRegression(train_df)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, pin_memory=False)

# Define Regression Model
class CrimeRiskRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CrimeRiskRegression, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.output_layer = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.output_layer(x)
        return torch.sigmoid(x)

# 🚀 Model Initialization (CPU Only)
device = torch.device("cpu")
model = CrimeRiskRegression(input_dim=6, hidden_dim=128).to(device)

# Loss Function & Optimizer
criterion = nn.HuberLoss(delta=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.002)

# Training Loop (Sequential Execution)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}] - Sequential Loss: {total_loss:.4f}')

# 🚀 End Timer
sequential_time = time.time() - start_time
print(f"🐢 Sequential Execution Time: {sequential_time:.2f} seconds")


Epoch [1/10] - Sequential Loss: 70.9535
Epoch [2/10] - Sequential Loss: 68.3578
Epoch [3/10] - Sequential Loss: 67.8143
Epoch [4/10] - Sequential Loss: 67.4693
Epoch [5/10] - Sequential Loss: 67.1701
Epoch [6/10] - Sequential Loss: 66.8987
Epoch [7/10] - Sequential Loss: 66.7001
Epoch [8/10] - Sequential Loss: 66.6154
Epoch [9/10] - Sequential Loss: 66.3425
Epoch [10/10] - Sequential Loss: 66.1101
🐢 Sequential Execution Time: 417.56 seconds


In [ ]:
# Convert predictions from 0-1 scale back to 1-5
def convert_predictions(preds):
    return (preds * 4) + 1  # Scale back to 1-5

# 🚀 Convert Predictions Back to 1-5 Scale (Sequential Execution)
model.eval()  # Set model to evaluation mode
sample_input = torch.tensor([[0.2, -1.1, 0.5, 0.8, 1.3, -0.5]], dtype=torch.float32).to(device)
output = model(sample_input)
converted_output = convert_predictions(output)

print(f"🐢 Raw Model Output (Sequential): {output.item():.4f}")
print(f"🐢 Converted Prediction (Sequential, 1-5 Scale): {converted_output.item():.2f}")

🐢 Raw Model Output (Sequential): 0.5066
🐢 Converted Prediction (Sequential, 1-5 Scale): 3.03
